In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Cell 1: Import required libraries
import os
import yaml
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

# load env variables
load_dotenv('../../../.env')

# Import your model and data loading components
from dataloader.dataset_wrapper import create_wrapper_from_dataframe

print("Libraries imported successfully!")

Libraries imported successfully!


In [3]:
# First install ClearML if not already installed: pip install clearml
import clearml
from clearml import Task, Logger

# Initialize ClearML Task
task = Task.init(project_name='CSMP_thesis_project', task_name='CSMP_traning_phase_1', reuse_last_task_id=False)
logger = Logger.current_logger()

ClearML Task: created new task id=3ca7b102c6c8472b9a7638bd224ecfae
ClearML results page: https://app.clear.ml/projects/0fec81950d384f0294d2c713df3887db/experiments/3ca7b102c6c8472b9a7638bd224ecfae/output/log


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


In [4]:
# Cell 2: Configuration and paths setup
CONFIG_PATH = "../../../configs/config.yaml"
TRAIN_CSV_PATH = "../../../data/traning_and_validation/train_deduplicated.csv"

# Device selection 
if torch.backends.mps.is_available():
    DEVICE = 'mps'
    print("Using MPS (Metal Performance Shaders) for GPU acceleration")
elif torch.cuda.is_available():
    DEVICE = 'cuda'
    print("Using CUDA for GPU acceleration")
else:
    DEVICE = 'cpu'
    print("Using CPU")

print(f"Using device: {DEVICE}")
print(f"Validation data path: {TRAIN_CSV_PATH}")

Using CUDA for GPU acceleration
Using device: cuda
Validation data path: ../../../data/traning_and_validation/train_deduplicated.csv


In [ ]:
# Cell 3: Load configuration
print("Loading configuration...")
config = yaml.load(open(CONFIG_PATH, "r"), Loader=yaml.FullLoader)

print("Configuration loaded:")
print(f"- Batch size: {config.get('batch_size', 64)}")
print(f"- Model config keys: {list(config.get('model', {}).keys())}")
print(f"- Loss config: {config.get('loss', {})}")

Loading configuration...
Configuration loaded:
- Batch size: 256
- Model config keys: []
- Loss config: {'temperature': 0.1, 'use_cosine_similarity': True, 'alpha_weight': 0.75}


In [6]:
task.connect(config)

{'batch_size': 256,
 'epochs': 10,
 'eval_every_n_epochs': 2,
 'log_every_n_steps': 2,
 'learning_rate': '1e-06',
 'weight_decay': 0.0001,
 'fp16_precision': True,
 'truncation': True,
 'model_config': {'emb_dim': 256,
  'spec_embed_dim': 256,
  'embed_dim': 128,
  'feat_dim': 512,
  'num_layer': 5,
  'layers': 5,
  'drop_ratio': 0.1,
  'dropout': 0.1,
  'pool': 'mean'},
 'dataset': {'s': 1,
  'num_workers': 0,
  'valid_size': 0.1,
  'ms2_file': '/home/xieting/graph_transformer_esa_hpc/data/qotf_20.mgf',
  'smi_file': '/home/xieting/graph_transformer_esa_hpc/data/smi_qtof_20.npy'},
 'loss': {'temperature': 0.1,
  'use_cosine_similarity': True,
  'alpha_weight': 0.75}}

In [7]:
# Cell 4: Load and explore validation data
print("Loading train data...")
df_train = pd.read_csv(TRAIN_CSV_PATH)

print(f"Validation dataset shape: {df_train.shape}")
print(f"Columns: {list(df_train.columns)}")
print(f"Sample data:")
df_train.head()

Loading train data...
Validation dataset shape: (798444, 10)
Columns: ['peaks_json', 'ion_source', 'compound_source', 'instrument', 'adduct', 'precursor_mz', 'smiles', 'inchikey', 'ion_mode', 'molecular_formula']
Sample data:


,peaks_json,ion_source,compound_source,instrument,adduct,precursor_mz,smiles,inchikey,ion_mode,molecular_formula
0,"[[42.014248, 0.10199999999999998], [42.26601, ...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
1,"[[49.01717, 0.155], [49.020023, 0.253], [67.05...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
2,"[[49.017338, 0.242], [49.020237, 0.181], [67.0...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
3,"[[49.01701, 0.144], [49.019947, 0.244], [139.0...",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3
4,"[[49.017166, 0.155], [49.020008, 0.253], [139....",ESI,Crude,Orbitrap,[M+H]+,377.186,CC12CCC(C(=O)N(CNc3cc4c(cc3)c3ccccc3o4)C1=O)C2...,RNKMIWQDRWSWCD-UHFFFAOYSA-N,Positive,C23H24N2O3


In [8]:
df_train_sample = df_train.sample(n=10000,random_state=42).reset_index(drop=True)

In [ ]:
# Cell 6: Prepare validation data loader
print("Preparing data loaders")

# Create data wrapper from DataFrame
wrapper = create_wrapper_from_dataframe(
    df=df_train_sample,
    batch_size=config.get('batch_size'),  
    num_workers=8,
    valid_size=config.get('valid_size'),  
    use_ddp=False,
    output_dir="../../../data/train_feature/",
    recompute=True
)

# Get the data loader
train_loader, val_loader = wrapper.get_data_loaders()

Preparing data loaders
Converting DataFrame to compatible files...
Processed 10000 valid spectra out of 10000 total entries.
Create data wrapper
calculating molecular graphs


  6%|▋         | 513/8000 [00:00<00:11, 629.28it/s]

SMILES [I-].O=C(OCC1=CC[N+]2(C)CCC(O)C12)C(O)(C(O)C)C(C)C calculation failure


 11%|█▏        | 901/8000 [00:01<00:12, 588.99it/s]

SMILES [Cl-].O=C(O)C=1C=CC=CC1C=2C=3C=CC(=CC3OC4=CC(C=CC42)=[N+](CC)CC)N(CC)CC calculation failure


...

SMILES [Cl-].OC=1C=C(O)C=2C=C(OC3OC(CO)C(O)C(O)C3O)C(=[O+]C2C1)C=4C=CC(O)=C(O)C4 calculation failure


 69%|██████▊   | 1373/2000 [00:02<00:01, 620.42it/s]

SMILES [Cl-].O=C1C2=CC=C(O)C(=C2OC(=C1C=3C=CC=4OCCCOC4C3)C)C[NH+](C)C calculation failure
SMILES [K+].O=S(=O)([O-])ON=C(SC1OC(CO)C(O)C(O)C1O)CC=C.O calculation failure


100%|██████████| 2000/2000 [00:03<00:00, 633.62it/s]

Calculated 1697 molecular graph-mass spectrometry pairs


In [10]:
from model import ModelCLR

# Initialize model architecture
model = ModelCLR(**config["model_config"]).to(DEVICE)

In [11]:
model

ModelCLR(
  (Smiles_model): SmilesModel(
    (x_embedding1): Embedding(119, 256)
    (x_embedding2): Embedding(4, 256)
    (x_embedding3): Embedding(8, 256)
    (x_embedding4): Embedding(6, 256)
    (x_embedding5): Embedding(5, 256)
    (gnns): ModuleList(
      (0-4): 5 x GINEConv()
    )
    (batch_norms): ModuleList(
      (0-4): 5 x BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feat_lin): Linear(in_features=256, out_features=512, bias=True)
    (out_lin): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=512, out_features=256, bias=True)
    )
  )
  (MS_model): MSModel(
    (mz_embedder): FourierEmbedder()
    (input_compress): Linear(in_features=257, out_features=256, bias=True)
    (peak_attn_layers): ModuleList(
      (0-4): 5 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in

In [12]:
# Parameter Count
print("PARAMETER ANALYSIS:")
print("-" * 40)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Trainable Parameters: {trainable_params:,}")


PARAMETER ANALYSIS:
----------------------------------------
Trainable Parameters: 6,102,784


In [13]:
# Memory Usage (approximate)
print("MEMORY ANALYSIS:")
print("-" * 40)
param_size = sum(p.numel() * p.element_size() for p in model.parameters())
buffer_size = sum(b.numel() * b.element_size() for b in model.buffers())
model_size_mb = (param_size + buffer_size) / 1024 / 1024

print(f"Model Size: {model_size_mb:.2f} MB")
print(f"Parameter Memory: {param_size / 1024 / 1024:.2f} MB")
print(f"Buffer Memory: {buffer_size / 1024 / 1024:.2f} MB")


MEMORY ANALYSIS:
----------------------------------------
Model Size: 23.29 MB
Parameter Memory: 23.28 MB
Buffer Memory: 0.01 MB


In [14]:
from loss.nt_xent import NTXentLoss

# Initialize loss function
temperature = config.get('loss', {}).get('temperature', 0.1)
batch_size = config.get('batch_size', 512)
use_cosine_similarity = config.get('loss', {}).get('use_cosine_similarity', True)
alpha_weight = config.get('loss', {}).get('alpha_weight', 1.0)

criterion = NTXentLoss(
    device=DEVICE, 
    batch_size=batch_size, 
    temperature=temperature, 
    use_cosine_similarity=use_cosine_similarity, 
    alpha_weight=alpha_weight
)

In [15]:
# Cell 15: Training Setup and Optimizer
import torch.optim as optim

print("Setting up training components...")
OUTPUT_DIR = "../../../models/models_experiments/candidate_v1"

# Initialize optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=float(config.get('learning_rate', 5e-6)),
    weight_decay=float(config.get('weight_decay', 1e-4))
)

# Training configuration
epochs = config.get('epochs', 100)
eval_every_n_epochs = config.get('eval_every_n_epochs', 5)
log_every_n_steps = config.get('log_every_n_steps', 2)

# Create checkpoint directory
checkpoint_dir = os.path.join(OUTPUT_DIR, "checkpoints")
os.makedirs(checkpoint_dir, exist_ok=True)

print(f"Training for {epochs} epochs")
print(f"Optimizer: AdamW with lr={config.get('learning_rate', 5e-6)}")
print(f"Checkpoint directory: {checkpoint_dir}")

Setting up training components...
Training for 10 epochs
Optimizer: AdamW with lr=1e-06
Checkpoint directory: ../../../models/models_experiments/candidate_v1/checkpoints


In [16]:
# Cell 16: Training and Evaluation Functions
def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    total_loss = 0
    num_batches = 0
    batch_losses = []
    
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}")
    
    for batch_idx, (graphs, mzs, intensities, num_peaks) in enumerate(progress_bar):
        
        # Move data to device
        graphs = graphs.to(device)
        mzs = mzs.to(device)
        intensities = intensities.to(device)
        num_peaks = num_peaks.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        mol_features, spec_features = model(graphs, mzs, intensities, num_peaks)
        loss = criterion(mol_features, spec_features)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Track loss
        batch_loss = loss.item()
        total_loss += batch_loss
        batch_losses.append(batch_loss)
        num_batches += 1
        
        # Update progress bar
        progress_bar.set_postfix({
            'Loss': f'{batch_loss:.4f}',
            'Avg Loss': f'{total_loss/num_batches:.4f}'
        })
        
        # Log every n steps
        if batch_idx % log_every_n_steps == 0:
            print(f"Epoch {epoch}, Batch {batch_idx}, Loss: {batch_loss:.4f}")
    
    avg_loss = total_loss / num_batches
    return avg_loss, batch_losses

In [17]:
def evaluate_model(model, val_loader, criterion, device, epoch):
    model.eval()
    total_loss = 0
    num_batches = 0
    
    molecular_features_list = []
    spectral_features_list = []
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc=f"Evaluating Epoch {epoch}")
        
        for batch_idx, (graphs, mzs, intensities, num_peaks) in enumerate(progress_bar):
            # Move data to device
            graphs = graphs.to(device)
            mzs = mzs.to(device)
            intensities = intensities.to(device)
            num_peaks = num_peaks.to(device)
            
            # Forward pass
            mol_features, spec_features = model(graphs, mzs, intensities, num_peaks)
            loss = criterion(mol_features, spec_features)
            
            total_loss += loss.item()
            num_batches += 1
            
            # Store features for retrieval metrics
            molecular_features_list.append(mol_features.cpu().numpy())
            spectral_features_list.append(spec_features.cpu().numpy())
            
            progress_bar.set_postfix({'Val Loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / num_batches
    
    # Compute retrieval metrics
    all_mol_features = np.vstack(molecular_features_list)
    all_spec_features = np.vstack(spectral_features_list)
    
    # Compute cosine similarities
    cosine_similarities = np.sum(all_mol_features * all_spec_features, axis=1)
    mean_similarity = np.mean(cosine_similarities)
        
    return avg_loss, mean_similarity

In [18]:
def save_checkpoint(model, optimizer, epoch, train_loss, val_loss, checkpoint_dir):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'config': config
    }
    
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pth')
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")
    
    # Save best model
    best_checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pth')
    if not os.path.exists(best_checkpoint_path):
        torch.save(checkpoint, best_checkpoint_path)
        print(f"Best model saved: {best_checkpoint_path}")
    else:
        best_checkpoint = torch.load(best_checkpoint_path)
        if val_loss < best_checkpoint['val_loss']:
            torch.save(checkpoint, best_checkpoint_path)
            print(f"New best model saved: {best_checkpoint_path}")

In [ ]:
import time
import json

# Training history for plotting
train_history = {
    'epochs': [],
    'train_losses': [],
    'val_losses': [],
    'val_similarities': [],
}

best_val_loss = float('inf')
start_time = time.time()

print("Starting training...")
print("=" * 60)

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    
    # Training phase
    train_loss, batch_losses = train_epoch(
        model, train_loader, criterion, optimizer, DEVICE, epoch
    )
    
    # Log training loss every epoch
    logger.report_scalar("Loss", "Train", iteration=epoch, value=train_loss)
    
    
    # Validation phase (every n epochs)
    if epoch % eval_every_n_epochs == 0:
        val_loss, val_similarity = evaluate_model(
            model, val_loader, criterion, DEVICE, epoch
        )
        
            
        # Log results
        epoch_time = time.time() - epoch_start_time
        total_time = time.time() - start_time
        
        # Log validation metrics to ClearML
        logger.report_scalar("Loss", "Validation", iteration=epoch, value=val_loss)
        logger.report_scalar("Similarity", "Cosine Similarity", iteration=epoch, value=val_similarity)
        
        print(f"\nEpoch {epoch}/{epochs}")
        print(f"Train Loss: {train_loss:.6f}")
        print(f"Val Loss: {val_loss:.6f}")
        print(f"Val mean similarity: {val_similarity:.4f}")
        print(f"Learning Rate: {current_lr:.2e}")
        print(f"Epoch Time: {epoch_time:.2f}s, Total Time: {total_time:.2f}s")
        print("-" * 60)
        
        # Store history - convert numpy types to Python native types
        train_history['epochs'].append(int(epoch))
        train_history['train_losses'].append(float(train_loss))
        train_history['val_losses'].append(float(val_loss))
        train_history['val_similarities'].append(float(val_similarity))
        
        # Save checkpoint
        save_checkpoint(
            model, optimizer, epoch, 
            train_loss, val_loss, checkpoint_dir
        )
        
        if epoch % (eval_every_n_epochs * 2) == 0 and len(train_history['epochs']) > 1:
            # Create loss plot
            import matplotlib.pyplot as plt
            
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
            
            # Loss plot
            epochs_list = train_history['epochs']
            ax1.plot(epochs_list, train_history['train_losses'], 'b-', label='Train Loss', linewidth=2)
            ax1.plot(epochs_list, train_history['val_losses'], 'r-', label='Val Loss', linewidth=2)
            ax1.set_xlabel('Epoch')
            ax1.set_ylabel('Loss')
            ax1.set_title('Training and Validation Loss')
            ax1.legend()
            ax1.grid(True, alpha=0.3)
            
            # Similarity plot
            ax2.plot(epochs_list, train_history['val_similarities'], 'g-', linewidth=2)
            ax2.set_xlabel('Epoch')
            ax2.set_ylabel('Cosine Similarity')
            ax2.set_title('Validation Cosine Similarity')
            ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            
            # Log plot to ClearML
            logger.report_matplotlib_figure("Training Progress", "Loss and Similarity", iteration=epoch, figure=plt)
            plt.close()
    
    else:
        current_lr = optimizer.param_groups[0]['lr']
        epoch_time = time.time() - epoch_start_time
        
        print(f"Epoch {epoch}/{epochs} - Train Loss: {train_loss:.6f}, LR: {current_lr:.2e}, Time: {epoch_time:.2f}s")

# Final summary logging
print("\nTraining completed!")
total_training_time = (time.time() - start_time)/3600
print(f"Total training time: {total_training_time:.2f} hours")

Starting training...


Training Epoch 1:   0%|          | 0/26 [00:00<?, ?it/s]

Training Epoch 1:   4%|▍         | 1/26 [00:00<00:23,  1.07it/s, Loss=5.6384, Avg Loss=5.6384]

Epoch 1, Batch 0, Loss: 5.6384


Training Epoch 1:  12%|█▏        | 3/26 [00:01<00:11,  2.09it/s, Loss=5.6191, Avg Loss=5.6229]

...

Training Epoch 10:  96%|█████████▌| 25/26 [00:07<00:00,  3.25it/s, Loss=5.5596, Avg Loss=5.5538]

Epoch 10, Batch 24, Loss: 5.5596


Evaluating Epoch 10: 100%|██████████| 7/7 [00:01<00:00,  6.01it/s, Val Loss=5.0849]



Epoch 10/10
Train Loss: 5.553835
Val Loss: 5.483869
Val mean similarity: -0.0431
Learning Rate: 1.00e-06
Epoch Time: 9.37s, Total Time: 163.79s
------------------------------------------------------------
Checkpoint saved: ../../../models/models_experiments/candidate_v1/checkpoints/checkpoint_epoch_10.pth
New best model saved: ../../../models/models_experiments/candidate_v1/checkpoints/best_model.pth

Training completed!
Total training time: 0.05 hours


TypeError: report_single_value() takes 3 positional arguments but 4 were given

In [ ]:
# Log final metrics
logger.report_single_value("Total Training Time (hours)", total_training_time)
logger.report_single_value("Best Validation Loss", min(train_history['val_losses']) if train_history['val_losses'] else float('inf'))
logger.report_single_value("Best Cosine Similarity", max(train_history['val_similarities']) if train_history['val_similarities'] else 0.0)
logger.report_single_value("Total Epochs", epochs)

# Close the task
task.close()